In [1]:
#Pre processing of the data
#Loading the dataset
import pandas as pd
df=pd.read_csv("D:/UOFG/SEM2/IV/Coursework/SystemC/imdb1000moviesinjune2022.csv")
df['Year'] = df['Year'].str.replace('-', '')
df['Year'] = df['Year'].str.replace('(', '')
df['Year'] = df['Year'].str.replace(')', '')
df['Year'] = df['Year'].str.replace('I', '')
df['GROSS'] = df['GROSS'].str.replace('M', '')
df['GROSS'] = df['GROSS'].str.replace('$', '')
df.dropna(inplace=True)
df['genre_new'] = df['genre'].str.split(',').str[0]
df['Year'] = df['Year'].astype(int)
df['GROSS'] = (df['GROSS'].astype(float)
                             .round(0)
                             .astype(int))
def categorize_year(Year):
    if Year >= 1900:
        decade_start = (Year // 10) * 10
        decade_end = decade_start + 9
        return f"{decade_start}-{decade_end}"
    else:
        return "Before 1900"

df['Decade'] = df['Year'].apply(categorize_year)
df[['Year', 'Decade']].head()

df.to_csv('D:/UOFG/SEM2/IV/Coursework/SystemC/modified_data.csv', index=False)

C:\Users\abhij\AppData\Local\Temp\ipykernel_18684\2538956216.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Year'] = df['Year'].str.replace('(', '')
C:\Users\abhij\AppData\Local\Temp\ipykernel_18684\2538956216.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Year'] = df['Year'].str.replace(')', '')
C:\Users\abhij\AppData\Local\Temp\ipykernel_18684\2538956216.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['GROSS'] = df['GROSS'].str.replace('$', '')


In [2]:
# System C
# System C

import altair as alt
import pandas as pd

# Load the dataset
df = pd.read_csv('D:/UOFG/SEM2/IV/Coursework/SystemC/modified_Data.csv')


df['genre_new_area'] = df['genre_new']


genre_stats = df.groupby('genre_new').agg(sum=('GROSS', 'sum'), mean=('GROSS', 'mean')).reset_index()

# Selection for brushing
brush = alt.selection_interval(encodings=['x'])


bar_chart = alt.Chart(genre_stats).mark_bar().encode(
    x=alt.X('genre_new:N', title='Genre', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('sum:Q', title='Total Gross Earnings (in million)'),
    color=alt.condition(brush, 'genre_new:N', alt.value('lightgray'), scale=alt.Scale(scheme='paired'))
).properties(
    title='Total Gross Earnings by Genre',
    width=800,  
    height=400  
).add_selection(
    brush  
)


line_chart = alt.Chart(genre_stats).mark_line(point=True).encode(
    x='genre_new:N',
    y=alt.Y('mean:Q', title='Average Gross Earnings (in million)'),
    color=alt.value('black') 
).transform_filter(
    brush 
).properties(
    width=800, 
    height=400 
)


final_chartss = alt.layer(bar_chart, line_chart).resolve_scale(
    y='independent' 
).properties(
    title='Total and Average Gross Earnings by Genre',
    width=800, 
    height=400  
)


total_gross = df.groupby('certificate')['GROSS'].sum().reset_index()

# Boxplot plot visualization
boxplot = alt.Chart(df).mark_boxplot().encode(
    x='certificate:N',
    y='GROSS:Q',
    color=alt.Color('certificate:N', scale=alt.Scale(scheme='paired')),
    tooltip=['certificate', 'GROSS']
).properties(
    title='Gross Earnings by Movie Certificate',
    width=800, 
    height=400  
)

# Density plot for gross earnings by certificate
area_chart = alt.Chart(df).mark_area().encode(
    alt.X('Decade:O', title='Year'),
    alt.Y('average(metascore):Q', title='Average Metascore'),
    color=alt.Color('genre_new_area:N', scale=alt.Scale(scheme='paired')),  # Use the duplicate field here
    tooltip=['Decade', 'average(metascore)', 'genre_new']
).transform_filter(
    brush 
).properties(
    title='Average Metascore by Genre Over Time',
    width=800,
    height=400
)


left_charts = alt.vconcat(final_chartss, area_chart ).resolve_scale(y='independent')
final_chart = alt.hconcat(left_charts, boxplot).resolve_scale(color='independent')
final_chart.save('D:/UOFG/SEM2/IV/Coursework/SystemC/sysC.html')


D:\Anaconda\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
